In [86]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
lr = LogisticRegression()
from sklearn.svm import SVC
svc = SVC()
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc, cohen_kappa_score
from sklearn.multiclass import OneVsRestClassifier

In [85]:
file = pd.read_excel('./LIWC2015 Results (training data_clean).xlsx')
#TF: TopicFeatures
#LF: boarder LinguisticFeatures
#LSF: LanguageSummaryFeatures, LIWC

**roc_auc value: micro**

In [3]:
#get the y_score for each classifier, the variable is the classifier type
#for svm and logisticRegression only
def get_y_score(method):
    classifier = OneVsRestClassifier(method)
    y_score = classifier.fit(X_train, y_train).decision_function(X_test)
    return y_score

In [4]:
#get a dictionary of micro roc_auc value
#micro: 
#the return value has the roc_value for each class, and the avg roc_auc value for all classes
def get_micro_auc(y_score):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:,i], y_score[:,i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    fpr['micro'], tpr['micro'], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc['micro'] = auc(fpr['micro'], tpr['micro'])
    return roc_auc

In [5]:
#For decision trees & Bayes
def get_y_score_for_tree(method):
    classifier = OneVsRestClassifier(method)
    y_score = classifier.fit(X_train, y_train).predict_proba(X_test)
    return y_score

In [78]:
#Feature selections

samplefile = file.sample(n = 800,replace=True) #take 800 as samples
    
# use those features and their combination as X to generate desireable result and write them down seperately 

X = samplefile.iloc[:,2:10] # only LIWC features-----LSF
   
#y = file['code']
# X = samplefile.iloc[:,10:43] # only Grammers features
#X = samplefile.iloc[:,43:79] # only Linguistic Feature
#X = samplefile.iloc[:,2:79] #LF + LSF
#X = samplefile.iloc[:,10:79] # Grammer + Linguistic = broader Linguistic features-----LF
#X = samplefile.iloc[:,79:] # only Topic Featurs----TF
#X = samplefile.iloc[:,43:] #Topic Features + Linguistic Features
#X = samplefile.iloc[:,2:] # all features combined----LSF + LF + TF
y = samplefile['code']
y = label_binarize(y, classes = [0,1,2,3,4])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
n_classes = y.shape[1]

In [28]:
#LogisticRegression
lr_y_score = get_y_score(lr)
lr_roc = get_micro_auc(lr_y_score)
print "Logistic Regression"
print lr_roc

Logistic Regression
{0: 0.898971247691902, 1: 0.7995151515151515, 2: 0.7440965915161876, 3: 0.8267336112163699, 4: 0.8259590636107366, 'micro': 0.8335677083333333}


In [29]:
#SVM
svm_y_score = get_y_score(svc)
svm_roc = get_micro_auc(svm_y_score)
print "SVM"
print svm_roc

SVM
{0: 0.9434186230546031, 1: 0.8898989898989899, 2: 0.9024468426113901, 3: 0.9054566123531641, 4: 0.886260571148425, 'micro': 0.9055468749999999}


In [30]:
#For decision trees
print "Decision Trees"
dtc_y_score = get_y_score_for_tree(dtc)
print get_micro_auc(dtc_y_score)

Decision Trees
{0: 0.7806647322606174, 1: 0.7587878787878788, 2: 0.8231114435302918, 3: 0.7961348995831754, 4: 0.7280916779017037, 'micro': 0.7848958333333333}


In [31]:
#For Naive Bayes
print "Naive Bayes"
gb_y_score = get_y_score_for_tree(gnb)
print get_micro_auc(gb_y_score)

Naive Bayes
{0: 0.7942495383803747, 1: 0.7767272727272727, 2: 0.5946148092744951, 3: 0.7248957938613112, 4: 0.6780242676798628, 'micro': 0.7244487847222223}


**kappa value**

In [33]:
def get_predict(method):
    classifier = OneVsRestClassifier(method)
    y_pred = classifier.fit(X_train, y_train).predict(X_test)
    return y_pred    

In [45]:
def get_kappa_score(y_pred):
    kappa = dict()
    for i in range(n_classes):
        kappa[i] = cohen_kappa_score(y_test[:,i], y_pred[:,i])
    return kappa

In [79]:
#LogisticRegression
lr_pred = get_predict(lr)
print "Logistic Regression"
print get_kappa_score(lr_pred)
print "Avg"
print np.mean(get_kappa_score(lr_pred).values())

Logistic Regression
{0: 0.0, 1: 0.13081081081081092, 2: 0.01670951156812328, 3: 0.0938098276962348, 4: 0.02751119641714661}
Avg
0.05376826929846312


In [80]:
#SVM
svm_pred = get_predict(svc)
print "SVM"
svm_kappa = get_kappa_score(svm_pred)
print svm_kappa
print "Avg"
print np.mean(svm_kappa.values())

SVM
{0: 0.6764408493427705, 1: 0.5151515151515151, 2: 0.6065036207132122, 3: 0.49320148331273184, 4: 0.5631067961165048}
Avg
0.5708808529273469


In [81]:
#Decision Trees
dtc_pred = get_predict(dtc)
print "Decision Trees"
dtc_kappa = get_kappa_score(dtc_pred)
print dtc_kappa
print "Avg"
print np.mean(dtc_kappa.values())

Decision Trees
{0: 0.5478723404255319, 1: 0.49211618257261414, 2: 0.5294616362865621, 3: 0.5525044495296212, 4: 0.5425459496255957}
Avg
0.532900111687985


In [82]:
#Bayes
gnb_pred = get_predict(gnb)
print "Bayes"
gnb_kappa = get_kappa_score(gnb_pred)
print gnb_kappa
print "Avg"
print np.mean(gnb_kappa.values())

Bayes
{0: 0.13842619184376792, 1: -0.0117647058823529, 2: 0.10802062102865373, 3: 0.2737588652482269, 4: 0.16422900125365658}
Avg
0.13453399469839045
